<a href="https://colab.research.google.com/github/suzannefox/burrow/blob/main/burrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup - install libraries and create functions

In [3]:
pip install palmerpenguins

In [4]:
import pandas as pd
import numpy as np
from palmerpenguins import load_penguins

In [5]:
# Supporting Functions

# return excel col equivalent of a numerical value
def excelcol(number):
  if number > 16384: return '16384 max'
  if number < 1: return 'must be > 0'

  letters = ""
  while number:
    number, remainder = divmod(number - 1, 26)
    letters = chr(65 + remainder) + letters

  return letters

# function to return minimum value for numeric variable or first alphabetical occurrence for string variable
def custom_min(variable):
  if pd.api.types.is_numeric_dtype(variable.dropna()):
    return variable.dropna().min()  # Drop NaNs and get the minimum value
  elif pd.api.types.is_string_dtype(variable.dropna()):
    return variable.dropna().sort_values(ascending=True).reset_index(drop=True)[0]  # Sort descending and get the first value
  else:
    return None

# function to return maximum value for numeric variable or last alphabetical occurrence for string variable
def custom_max(variable):
  if pd.api.types.is_numeric_dtype(variable.dropna()):
    return variable.dropna().max()  # Drop NaNs and get the minimum value
  elif pd.api.types.is_string_dtype(variable.dropna()):
    return variable.dropna().sort_values(ascending=False).reset_index(drop=True)[0]  # Sort and get the first alphabetical value
  else:
    return None

# return mean if a variable is numeric, return '-' otherwise
def custom_mean(variable):
  if pd.api.types.is_numeric_dtype(variable.dropna()):
    return variable.dropna().mean().round(1)
  else:
    return '-'

# return skew if a variable is numeric, return '-' otherwise
def custom_skew(variable):
  if pd.api.types.is_numeric_dtype(variable.dropna()):
    return variable.dropna().skew().round(1)
  else:
    return '-'

# determine a more explicit data type
def custom_type(variable):
  if pd.api.types.is_numeric_dtype(variable.dropna()):
    return('numeric')
  elif pd.api.types.is_string_dtype(variable.dropna()):
    return('string')
  elif pd.api.types.is_bool_dtype(variable.dropna()):
    return('bool')
  elif pd.api.types.is_datetime64_dtype(variable.dropna()):
    return('datetime')
  elif pd.api.types.is_object_dtype(variable.dropna()):
    return('object')
  elif pd.api.types.is_integer_dtype(variable.dropna()):
    return('integer')
  elif pd.api.types.is_float_dtype(variable.dropna()):
    return('float')
  elif pd.api.types.is_categorical_dtype(variable.dropna()):
    return('categorical')
  elif pd.api.types.is_sparse_dtype(variable.dropna()):
    return('sparse')
  elif pd.api.types.is_generic_dtype(variable.dropna()):
    return('generic')
  elif pd.api.types.is_complex_dtype(variable.dropna()):
    return('complex')
  else:
    return None


In [6]:
# for a supplied dataframe (df) return a dataframe of metadata
# optionally supply a second dataframe (catalog) with variable name and
# any user supplied information such as description or notes
def get_metadata(df, catalog=None, verbose=True):

  if df.empty:
    print('... dataframe is empty')
    return

  if verbose:
    df_name = [name for name in globals() if globals()[name] is df][0]
    print(f'... dataframe {df_name} has {df.shape[0]} records and {df.shape[1]} variables')

  # Main Body, make a dataframe to return
  dfout = pd.DataFrame({'Variables': df.columns})

  # order of the variables and their spreadsheet column
  dfout['Order'] = range(1, df.shape[1] + 1)
  dfout['Excel'] = dfout['Order'].apply(excelcol)

  # add catalog if there is one
  if catalog is not None:
    if 'Variables' not in catalog.columns:
      print(" ... WARNING: catalog dataframe missing a 'Variables' column, ignoring")
    else:
      dfout = dfout.merge(catalog, on='Variables', how='left').fillna('')

  # variable class and more specific type
  dfout['Class'] = df.dtypes.astype(str).values
  dfout['Type'] = df.apply(custom_type).reset_index(drop=True)

  # NAs, count and percentage
  dfout['NA_Tot'] = df.isna().sum().reset_index(drop=True)
  dfout['NA_PC'] = (df.isna().sum().reset_index(drop=True) / len(df) * 100).apply(lambda x: f"{x:.0f}%")

  # unique values, count and percentage
  dfout['Unique_Tot'] = df.nunique().reset_index(drop=True)
  dfout['Unique_PC'] = (df.nunique().reset_index(drop=True) / len(df) * 100).apply(lambda x: f"{x:.0f}%")

  # min and max values for numeric columns, or alphabetic values for strings
  dfout['Min'] = df.apply(custom_min).reset_index(drop=True)
  dfout['Max'] = df.apply(custom_max).reset_index(drop=True)

  # for numerics only, mean and skew of the distribution
  dfout['Mean'] = df.apply(custom_mean).reset_index(drop=True)
  dfout['Skew'] = df.apply(custom_skew).reset_index(drop=True)

  return dfout

# Penguins data example

In [7]:
penguins = load_penguins()

In [9]:
catalog = pd.DataFrame({'Variables': ['species','sex','island'],
                        'catalog': ['Penguin Species','Gender','Island they live on']})

get_metadata(penguins, catalog = catalog)

... dataframe penguins has 344 records and 8 variables


,Variables,Order,Excel,catalog,Class,Type,NA_Tot,NA_PC,Unique_Tot,Unique_PC,Min,Max,Mean,Skew
0,species,1,A,Penguin Species,object,string,0,0%,3,1%,Adelie,Gentoo,-,-
1,island,2,B,Island they live on,object,string,0,0%,3,1%,Biscoe,Torgersen,-,-
2,bill_length_mm,3,C,,float64,numeric,2,1%,164,48%,32.1,59.6,43.9,0.1
3,bill_depth_mm,4,D,,float64,numeric,2,1%,80,23%,13.1,21.5,17.2,-0.1
4,flipper_length_mm,5,E,,float64,numeric,2,1%,55,16%,172.0,231.0,200.9,0.3
5,body_mass_g,6,F,,float64,numeric,2,1%,94,27%,2700.0,6300.0,4201.8,0.5
6,sex,7,G,Gender,object,string,11,3%,2,1%,female,male,-,-
7,year,8,H,,int64,numeric,0,0%,3,1%,2007,2009,2008.0,-0.1
